In [ ]:
#Importing relevant Python libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#A function for extracting interaction energetic values from a data file
#Input: the path of the data file
#Output: a dataframe with the energetic values, each contained within a specific column according to the correpsonding energy term (Coulomb, Van der Waals (LJ))
def Generate_Interaction_Energies_df(file_name):
    
    #Reading the data file as lists
    y = np.loadtxt(file_name, comments = ['@', '#'], unpack = True)
    lists = []
    for i in range(len(y)):
        lists.append(list(y[i]))

    #Inserting the lists to a dictionary, which would be converted to a dataframe with suitable column names, according to the energy term
    d = {}
    for i in range(len(lists)):
        d[i] = lists[i]
    df = pd.DataFrame(d)
    df.set_index(0, inplace=True)
    if len(df.columns) == 4:
        df.columns = ['PD Coul-SR', 'PD LJ-SR', 'PD Coul-14', 'PD LJ-14']
    elif len(df.columns) == 12:
        df.columns = ['PP Coul-SR', 'PP LJ-SR', 'PP Coul-14', 'PP LJ-14', 'PD Coul-SR', 'PD LJ-SR', 'PD Coul-14', 'PD LJ-14', 'DD Coul-SR', 'DD LJ-SR', 'DD Coul-14', 'DD LJ-14']

    #Returning the dataframe    
    return df

#A function for convering multiple interaction energy data files to multiple corresponding dataframes, utilizing the above function
#Input: a list with the paths of the data files
#Output: a list with the dataframes of the energetic values, each contained within a specific column according to the correpsonding energy term (Coulomb, Van der Waals (LJ))
def from_files_to_dfs(two_files_list):

    #Utilizing the above function, converting each energy file in the input list to a corresponding dataframe, stored in an output list of dataframes
    dfs_list = []
    for i in range(len(two_files_list)):
        dfs_list.append(Generate_Interaction_Energies_df(two_files_list[i]))

    #Returning the output list of dataframes
    return dfs_list

In [ ]:
#A function for averaging two bending measurements, in case bending angle was measured for a certain distance window (see Materials and Methods) on two DNA regions - they have to be avergaed.
#Input: a list of file paths containing all bending angle measurements taken for one DNA region, a list of file paths containing all bending angle measurements taken for second DNA region
#Output: a list of file paths containing all bending angle measurements averaged for the two DNA regions
def average_two_bending_measures(first_bending_measure_files, second_bending_measure_files):

    #Defining the output list, initializing it as an empty list
    averaged_bending_measure_copies = []

    #Running over each bending angle file in the two input lists
    for i in range(len(first_bending_measure_files)):

        #Reading the data from the current input files of both lists
        first_bending_measure_curr_copy = list(np.loadtxt(first_bending_measure_files[i], comments = ['@', '#', '&'], unpack = True)[1])
        second_bending_measure_curr_copy = list(np.loadtxt(second_bending_measure_files[i], comments = ['@', '#', '&'], unpack = True)[1])

        #Adding the averaged bending angle list to the output list
        averaged_bending_measure_copies.append([(first_bending_measure_curr_copy[x] + second_bending_measure_curr_copy[x]) / 2 for x in range(len(first_bending_measure_curr_copy))])
    
    #Returning the ouput list
    return averaged_bending_measure_copies

#A function for creating all relevant plots to Hfq interaction with different DNA sequences, bearing different bendability rates
#Input: list of files containing the values of Hfq interaction energy with first type of DNA sequences (denoted as AT), the column in the first (AT) file tables containing the electrostatic interaction energy value, list of files containing the values of Hfq interaction energy with second type of DNA sequences (denoted as GC), the column in the second (GC) file tables containing the electrostatic interaction energy value, a list of files containing the bending angle (for a certain distance window, see Materials and Methods) values of the first DNA type, , a list of files containing the bending angle (for a certain distance window, see Materials and Methods) values of the second DNA type, a boolean specifying whether the current distance window requires averaging between two DNA regions or not, a title for the plots specifying the DNA type and the distance window, the plot color of the first DNA, the plot color of the second DNA, the plot label of the first DNA, the plot label of the second DNA
#Output: None
def create_relevant_plots(AT_energy_files, AT_Coulomb_col, GC_energy_files, GC_Coulomb_col, AT_bending_files, GC_bending_files, averaged_bending_measures, title, AT_color = 'mediumpurple', GC_color = 'sienna', AT_label = 'AT', GC_label = 'GC'):

    #Initializing the list of all energy values copies of the first DNA to be empty
    all_AT_energies = []

    #Reading the energy files of the first DNA to lists stored within the initialized list
    for AT_energy_file in AT_energy_files:
        loaded_AT_energy_file = list(np.loadtxt(AT_energy_file, comments = ['@', '#', '&'], unpack = True)[AT_Coulomb_col])
        for i in range(len(loaded_AT_energy_file)):
            all_AT_energies.append(loaded_AT_energy_file[i])
    
    #Initializing the list of all energy values copies of the second DNA to be empty
    all_GC_energies = []

    #Reading the energy files of the second DNA to lists stored within the initialized list
    for GC_energy_file in GC_energy_files:
        loaded_GC_energy_file = list(np.loadtxt(GC_energy_file, comments = ['@', '#', '&'], unpack = True)[GC_Coulomb_col])
        for i in range(len(loaded_GC_energy_file)):
            all_GC_energies.append(loaded_GC_energy_file[i])

    #Initializing the list of all bending angle values copies of the first DNA to be empty
    all_AT_angles = []

    #Reading the bending angle files of the first DNA to lists stored within the initialized list, taking into account whether the bending of two DNA regions should be averaged or not
    for AT_angles_file in AT_bending_files:
        if averaged_bending_measures:
            loaded_AT_angles_file = AT_angles_file.copy()
        else:
            loaded_AT_angles_file = list(np.loadtxt(AT_angles_file, comments = ['@', '#', '&'], unpack = True)[1])
        for i in range(len(loaded_AT_angles_file)):
            all_AT_angles.append(loaded_AT_angles_file[i])
    
    #Initializing the list of all bending angle values copies of the second DNA to be empty
    all_GC_angles = []

    #Reading the bending angle files of the second DNA to lists stored within the initialized list, taking into account whether the bending of two DNA regions should be averaged or not
    for GC_angles_file in GC_bending_files:
        if averaged_bending_measures:
            loaded_GC_angles_file = GC_angles_file.copy()
        else:
            loaded_GC_angles_file = list(np.loadtxt(GC_angles_file, comments = ['@', '#', '&'], unpack = True)[1])
        for i in range(len(loaded_GC_angles_file)):
            all_GC_angles.append(loaded_GC_angles_file[i])
    
    #Plotting 1D density plots of the energies of both DNA types - Panel S3
    plt.figure(figsize = (7, 7))
    plt.rcParams['pdf.fonttype'] = 42
    sns.kdeplot(all_AT_energies, color = AT_color, label = AT_label)
    sns.kdeplot(all_GC_energies, color = GC_color, label = GC_label, alpha = 0.65)
    plt.legend()
    plt.title(title)
    plt.savefig('energy_densities_1D_' + title + '.pdf', format = 'pdf')

    #Plotting 1D density plots of the bending angles of both DNA types - Panel S2
    plt.figure(figsize = (7, 7))
    plt.rcParams['pdf.fonttype'] = 42
    sns.kdeplot(all_AT_angles, color = AT_color, label = AT_label)
    sns.kdeplot(all_GC_angles, color = GC_color, label = GC_label, alpha = 0.65)
    plt.legend()
    plt.title(title)
    plt.savefig('angles_densities_1D_' + title + '.pdf', format = 'pdf')

    #Plotting 2D density plots of the energies vs. the bending angles of both DNA types - Panel 4B and Panel S3
    plt.figure(figsize = (7, 7))
    plt.rcParams['pdf.fonttype'] = 42
    sns.kdeplot(x = all_AT_angles, y = all_AT_energies, color = AT_color, label = AT_label)
    sns.kdeplot(x = all_GC_angles, y = all_GC_energies, color = GC_color, label = GC_label, alpha = 0.65)
    plt.legend()
    plt.title(title)
    plt.savefig('angles_vs_energies_densities_2D_' + title + '.pdf', format = 'pdf')

#Defining prefix of both dDDD and X12 DNA types (both AT-rich and GC-rich) - for reading the file - SPECIFY THE PATH TO THE DIRECTORY CONTAINING THE DATA FILES INSTEAD
AT_DDD_prefix = '/trajectories/yoav/Hfq_DDD_NMG_3_replicates/'; GC_DDD_prefix = '/trajectories/yoav/Hfq_DDD_GGCC_300ns'; AT_X12_prefix = '/trajectories/yoav/Hfq_B_DNA_AT_NoPR/'; GC_X12_prefix = '/trajectories/yoav/Hfq_B_DNA_GC/'

#Specifying the paths of the energy and bending angle files of both dDDD DNA types (AT-rich and GC-rich) - SPECIFY THE NAMES OF THE DATA FILES INSTEAD
AT_DDD_energy_files = ["/home_b/yoav/Hfq_DDD_NMG_24_3/Hfq_DDD_NMG_3_200ns_first_replicate_energy.xvg", "/home_b/yoav/Hfq_DDD_NMG_24_3/Hfq_DDD_NMG_3_200ns_second_replicate_energy.xvg", "/home_b/yoav/Hfq_DDD_NMG_24_3/Hfq_DDD_NMG_3_200ns_third_replicate_energy.xvg"]; GC_DDD_energy_files = ["/home_b/yoav/Hfq_DDD_GGCC/Hfq_DDD_GGCC_300ns_1_energy.xvg", "/home_b/yoav/Hfq_DDD_GGCC/Hfq_DDD_GGCC_300ns_2_energy.xvg", "/home_b/yoav/Hfq_DDD_GGCC/Hfq_DDD_GGCC_300ns_3_energy.xvg"]
AT_DDD_bending_2_23_files = [AT_DDD_prefix + 'bending_angle_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_2_23_files = [GC_DDD_prefix + '/bending_angle_DDD_GGCC_300ns_I.xvg', GC_DDD_prefix + '_2/bending_angle_DDD_GGCC_300ns_II.xvg', GC_DDD_prefix + '_3/bending_angle_DDD_GGCC_300ns_III.xvg']
AT_DDD_bending_5_8_files = [AT_DDD_prefix + 'bending_angle_local_first_AATT_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_local_first_AATT_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_local_first_AATT_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_5_8_files = [GC_DDD_prefix + '/bending_angle_FIRST_GGCC_DDD_GGCC_300ns_I.xvg', GC_DDD_prefix + '_2/bending_angle_FIRST_GGCC_DDD_GGCC_300ns_II.xvg', GC_DDD_prefix + '_3/bending_angle_FIRST_GGCC_DDD_GGCC_300ns_III.xvg']
AT_DDD_bending_17_20_files = [AT_DDD_prefix + 'bending_angle_local_second_AATT_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_local_second_AATT_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_local_second_AATT_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_17_20_files = [GC_DDD_prefix + '/bending_angle_SECOND_GGCC_DDD_GGCC_300ns_I.xvg', GC_DDD_prefix + '_2/bending_angle_SECOND_GGCC_DDD_GGCC_300ns_II.xvg', GC_DDD_prefix + '_3/bending_angle_SECOND_GGCC_DDD_GGCC_300ns_III.xvg']
AT_DDD_bending_5_17_files = [AT_DDD_prefix + 'bending_angle_5-17_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_5-17_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_5-17_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_5_17_files = [GC_DDD_prefix + '/bending_angle_5-17_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_5-17_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_5-17_DDD_GGCC_replicates_III.xvg']
AT_DDD_bending_8_20_files = [AT_DDD_prefix + 'bending_angle_8-20_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_8-20_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_8-20_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_8_20_files = [GC_DDD_prefix + '/bending_angle_8-20_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_8-20_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_8-20_DDD_GGCC_replicates_III.xvg']
AT_DDD_bending_5_20_files = [AT_DDD_prefix + 'bending_angle_5-20_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_5-20_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_5-20_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_5_20_files = [GC_DDD_prefix + '/bending_angle_5-20_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_5-20_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_5-20_DDD_GGCC_replicates_III.xvg']
AT_DDD_bending_2_11_files = [AT_DDD_prefix + 'bending_angle_2-11_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_2-11_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_2-11_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_2_11_files = [GC_DDD_prefix + '/bending_angle_2-11_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_2-11_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_2-11_DDD_GGCC_replicates_III.xvg']
AT_DDD_bending_3_10_files = [AT_DDD_prefix + 'bending_angle_3-10_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_3-10_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_3-10_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_3_10_files = [GC_DDD_prefix + '/bending_angle_3-10_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_3-10_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_3-10_DDD_GGCC_replicates_III.xvg']
AT_DDD_bending_4_9_files = [AT_DDD_prefix + 'bending_angle_4-9_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_4-9_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_4-9_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_4_9_files = [GC_DDD_prefix + '/bending_angle_4-9_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_4-9_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_4-9_DDD_GGCC_replicates_III.xvg']
AT_DDD_bending_14_23_files = [AT_DDD_prefix + 'bending_angle_14-23_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_14-23_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_14-23_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_14_23_files = [GC_DDD_prefix + '/bending_angle_14-23_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_14-23_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_14-23_DDD_GGCC_replicates_III.xvg']
AT_DDD_bending_15_22_files = [AT_DDD_prefix + 'bending_angle_15-22_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_15-22_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_15-22_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_15_22_files = [GC_DDD_prefix + '/bending_angle_15-22_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_15-22_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_15-22_DDD_GGCC_replicates_III.xvg']
AT_DDD_bending_16_21_files = [AT_DDD_prefix + 'bending_angle_16-21_DDD_NMG_replicates_I.xvg', AT_DDD_prefix + 'bending_angle_16-21_DDD_NMG_replicates_II.xvg', AT_DDD_prefix + 'bending_angle_16-21_DDD_NMG_replicates_III.xvg']; GC_DDD_bending_16_21_files = [GC_DDD_prefix + '/bending_angle_16-21_DDD_GGCC_replicates_I.xvg', GC_DDD_prefix + '_2/bending_angle_16-21_DDD_GGCC_replicates_II.xvg', GC_DDD_prefix + '_3/bending_angle_16-21_DDD_GGCC_replicates_III.xvg']

#Averaging the bending angles of two DNA regions for specific distance windows, when required - for both dDDD DNA types (AT-rich and GC-rich)
AT_DDD_bending_unified_4_mer_files = average_two_bending_measures(AT_DDD_bending_5_8_files, AT_DDD_bending_17_20_files)
GC_DDD_bending_unified_4_mer_files = average_two_bending_measures(GC_DDD_bending_5_8_files, GC_DDD_bending_17_20_files)
AT_DDD_bending_unified_6_mer_files = average_two_bending_measures(AT_DDD_bending_4_9_files, AT_DDD_bending_16_21_files)
GC_DDD_bending_unified_6_mer_files = average_two_bending_measures(GC_DDD_bending_4_9_files, GC_DDD_bending_16_21_files)
AT_DDD_bending_unified_8_mer_files = average_two_bending_measures(AT_DDD_bending_3_10_files, AT_DDD_bending_15_22_files)
GC_DDD_bending_unified_8_mer_files = average_two_bending_measures(GC_DDD_bending_3_10_files, GC_DDD_bending_15_22_files)
AT_DDD_bending_unified_10_mer_files = average_two_bending_measures(AT_DDD_bending_2_11_files, AT_DDD_bending_14_23_files)
GC_DDD_bending_unified_10_mer_files = average_two_bending_measures(GC_DDD_bending_2_11_files, GC_DDD_bending_14_23_files)
AT_DDD_bending_unified_13_mer_files = average_two_bending_measures(AT_DDD_bending_5_17_files, AT_DDD_bending_8_20_files)
GC_DDD_bending_unified_13_mer_files = average_two_bending_measures(GC_DDD_bending_5_17_files, GC_DDD_bending_8_20_files)

#Specifying the paths of the energy and bending angle files of both X12 DNA types (AT-rich and GC-rich) - SPECIFY THE NAMES OF THE DATA FILES INSTEAD
AT_X12_energy_files = ["/home_b/yoav/Hfq_B_DNA_AT/Hfq_B_DNA_ATX12_energy_NoPR_I.xvg", "/home_b/yoav/Hfq_B_DNA_AT/Hfq_B_DNA_ATX12_energy_NoPR_II.xvg", "/home_b/yoav/Hfq_B_DNA_AT/Hfq_B_DNA_ATX12_energy_NoPR_III.xvg"]; GC_X12_energy_files = ["/home_b/yoav/Hfq_B_DNA_GC/Hfq_B_DNA_GCX12.energy.I.xvg", "/home_b/yoav/Hfq_B_DNA_GC/Hfq_B_DNA_GCX12.energy.II.xvg", "/home_b/yoav/Hfq_B_DNA_GC/Hfq_B_DNA_GCX12.energy.III.xvg"]
AT_X12_bending_2_23_files = [AT_X12_prefix + 'bending_angle_ATX12_NoPosRes_I.xvg', AT_X12_prefix + 'bending_angle_ATX12_NoPosRes_II.xvg', AT_X12_prefix + 'bending_angle_ATX12_NoPosRes_I.xvg']; GC_X12_bending_2_23_files = [GC_X12_prefix + '/bending_angle_GCX12_NoPosRes_I.xvg', GC_X12_prefix + '/bending_angle_GCX12_NoPosRes_II.xvg', GC_X12_prefix + '/bending_angle_GCX12_NoPosRes_III.xvg']
AT_X12_bending_5_8_files = [AT_X12_prefix + 'bending_angle_first_stratch_ATX12_NoPosRes_I.xvg', AT_X12_prefix + 'bending_angle_first_stratch_ATX12_NoPosRes_II.xvg', AT_X12_prefix + 'bending_angle_first_stratch_ATX12_NoPosRes_III.xvg']; GC_X12_bending_5_8_files = [GC_X12_prefix + '/bending_angle_first_stratch_GCX12_NoPosRes_I.xvg', GC_X12_prefix + '/bending_angle_first_stratch_GCX12_NoPosRes_II.xvg', GC_X12_prefix + '/bending_angle_first_stratch_GCX12_NoPosRes_III.xvg']
AT_X12_bending_17_20_files = [AT_X12_prefix + 'bending_angle_second_stratch_ATX12_NoPosRes_I.xvg', AT_X12_prefix + 'bending_angle_second_stratch_ATX12_NoPosRes_II.xvg', AT_X12_prefix + 'bending_angle_second_stratch_ATX12_NoPosRes_III.xvg']; GC_X12_bending_17_20_files = [GC_X12_prefix + '/bending_angle_second_stratch_GCX12_NoPosRes_I.xvg', GC_X12_prefix + '/bending_angle_second_stratch_GCX12_NoPosRes_II.xvg', GC_X12_prefix + '/bending_angle_second_stratch_GCX12_NoPosRes_III.xvg']
AT_X12_bending_5_17_files = [AT_X12_prefix + 'bending_angle_5-17_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_5-17_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_5-17_ATX12_III.xvg']; GC_X12_bending_5_17_files = [GC_X12_prefix + '/bending_angle_5-17_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_5-17_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_5-17_GCX12_III.xvg']
AT_X12_bending_8_20_files = [AT_X12_prefix + 'bending_angle_8-20_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_8-20_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_8-20_ATX12_III.xvg']; GC_X12_bending_8_20_files = [GC_X12_prefix + '/bending_angle_8-20_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_8-20_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_8-20_GCX12_III.xvg']
AT_X12_bending_5_20_files = [AT_X12_prefix + 'bending_angle_5-20_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_5-20_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_5-20_ATX12_III.xvg']; GC_X12_bending_5_20_files = [GC_X12_prefix + '/bending_angle_5-20_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_5-20_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_5-20_GCX12_III.xvg']
AT_X12_bending_2_11_files = [AT_X12_prefix + 'bending_angle_2-11_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_2-11_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_2-11_ATX12_III.xvg']; GC_X12_bending_2_11_files = [GC_X12_prefix + '/bending_angle_2-11_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_2-11_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_2-11_GCX12_III.xvg']
AT_X12_bending_3_10_files = [AT_X12_prefix + 'bending_angle_3-10_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_3-10_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_3-10_ATX12_III.xvg']; GC_X12_bending_3_10_files = [GC_X12_prefix + '/bending_angle_3-10_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_3-10_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_3-10_GCX12_III.xvg']
AT_X12_bending_4_9_files = [AT_X12_prefix + 'bending_angle_4-9_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_4-9_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_4-9_ATX12_III.xvg']; GC_X12_bending_4_9_files = [GC_X12_prefix + '/bending_angle_4-9_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_4-9_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_4-9_GCX12_III.xvg']
AT_X12_bending_14_23_files = [AT_X12_prefix + 'bending_angle_14-23_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_14-23_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_14-23_ATX12_III.xvg']; GC_X12_bending_14_23_files = [GC_X12_prefix + '/bending_angle_14-23_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_14-23_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_14-23_GCX12_III.xvg']
AT_X12_bending_15_22_files = [AT_X12_prefix + 'bending_angle_15-22_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_15-22_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_15-22_ATX12_III.xvg']; GC_X12_bending_15_22_files = [GC_X12_prefix + '/bending_angle_15-22_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_15-22_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_15-22_GCX12_III.xvg']
AT_X12_bending_16_21_files = [AT_X12_prefix + 'bending_angle_16-21_ATX12_I.xvg', AT_X12_prefix + 'bending_angle_16-21_ATX12_II.xvg', AT_X12_prefix + 'bending_angle_16-21_ATX12_III.xvg']; GC_X12_bending_16_21_files = [GC_X12_prefix + '/bending_angle_16-21_GCX12_I.xvg', GC_X12_prefix + '/bending_angle_16-21_GCX12_II.xvg', GC_X12_prefix + '/bending_angle_16-21_GCX12_III.xvg']

#Averaging the bending angles of two DNA regions for specific distance windows, when required - for both X12 DNA types (AT-rich and GC-rich)
AT_X12_bending_unified_4_mer_files = average_two_bending_measures(AT_X12_bending_5_8_files, AT_X12_bending_17_20_files)
GC_X12_bending_unified_4_mer_files = average_two_bending_measures(GC_X12_bending_5_8_files, GC_X12_bending_17_20_files)
AT_X12_bending_unified_6_mer_files = average_two_bending_measures(AT_X12_bending_4_9_files, AT_X12_bending_16_21_files)
GC_X12_bending_unified_6_mer_files = average_two_bending_measures(GC_X12_bending_4_9_files, GC_X12_bending_16_21_files)
AT_X12_bending_unified_8_mer_files = average_two_bending_measures(AT_X12_bending_3_10_files, AT_X12_bending_15_22_files)
GC_X12_bending_unified_8_mer_files = average_two_bending_measures(GC_X12_bending_3_10_files, GC_X12_bending_15_22_files)
AT_X12_bending_unified_10_mer_files = average_two_bending_measures(AT_X12_bending_2_11_files, AT_X12_bending_14_23_files)
GC_X12_bending_unified_10_mer_files = average_two_bending_measures(GC_X12_bending_2_11_files, GC_X12_bending_14_23_files)
AT_X12_bending_unified_13_mer_files = average_two_bending_measures(AT_X12_bending_5_17_files, AT_X12_bending_8_20_files)
GC_X12_bending_unified_13_mer_files = average_two_bending_measures(GC_X12_bending_5_17_files, GC_X12_bending_8_20_files)

#Creating all relevant plots for the AT-rich DNA sequences (dDDD and X12) using the defined function
create_relevant_plots(AT_DDD_energy_files, 5, AT_X12_energy_files, 1, AT_DDD_bending_2_23_files, AT_X12_bending_2_23_files, False, 'AT_22_mer', 'mediumpurple', 'hotpink', 'AT DDD', 'AT X12')
create_relevant_plots(AT_DDD_energy_files, 5, AT_X12_energy_files, 1, AT_DDD_bending_unified_4_mer_files, AT_X12_bending_unified_4_mer_files, True, 'AT_4_mer', 'mediumpurple', 'hotpink', 'AT DDD', 'AT X12')
create_relevant_plots(AT_DDD_energy_files, 5, AT_X12_energy_files, 1, AT_DDD_bending_unified_6_mer_files, AT_X12_bending_unified_6_mer_files, True, 'AT_6_mer', 'mediumpurple', 'hotpink', 'AT DDD', 'AT X12')
create_relevant_plots(AT_DDD_energy_files, 5, AT_X12_energy_files, 1, AT_DDD_bending_unified_8_mer_files, AT_X12_bending_unified_8_mer_files, True, 'AT_8_mer', 'mediumpurple', 'hotpink', 'AT DDD', 'AT X12')
create_relevant_plots(AT_DDD_energy_files, 5, AT_X12_energy_files, 1, AT_DDD_bending_unified_10_mer_files, AT_X12_bending_unified_10_mer_files, True, 'AT_10_mer', 'mediumpurple', 'hotpink', 'AT DDD', 'AT X12')
create_relevant_plots(AT_DDD_energy_files, 5, AT_X12_energy_files, 1, AT_DDD_bending_unified_13_mer_files, AT_X12_bending_unified_13_mer_files, True, 'AT_13_mer', 'mediumpurple', 'hotpink', 'AT DDD', 'AT X12')
create_relevant_plots(AT_DDD_energy_files, 5, AT_X12_energy_files, 1, AT_DDD_bending_5_20_files, AT_X12_bending_5_20_files, False, 'AT_16_mer', 'mediumpurple', 'hotpink', 'AT DDD', 'AT X12')

#Creating all relevant plots for the GC-rich DNA sequences (dDDD and X12) using the defined function
create_relevant_plots(GC_DDD_energy_files, 5, GC_X12_energy_files, 5, GC_DDD_bending_2_23_files, GC_X12_bending_2_23_files, False, 'GC_22_mer', 'sienna', 'darkorange', 'GC DDD', 'GC X12')
create_relevant_plots(GC_DDD_energy_files, 5, GC_X12_energy_files, 5, GC_DDD_bending_unified_4_mer_files, GC_X12_bending_unified_4_mer_files, True, 'GC_4_mer', 'sienna', 'darkorange', 'GC DDD', 'GC X12')
create_relevant_plots(GC_DDD_energy_files, 5, GC_X12_energy_files, 5, GC_DDD_bending_unified_6_mer_files, GC_X12_bending_unified_6_mer_files, True, 'GC_6_mer', 'sienna', 'darkorange', 'GC DDD', 'GC X12')
create_relevant_plots(GC_DDD_energy_files, 5, GC_X12_energy_files, 5, GC_DDD_bending_unified_8_mer_files, GC_X12_bending_unified_8_mer_files, True, 'GC_8_mer', 'sienna', 'darkorange', 'GC DDD', 'GC X12')
create_relevant_plots(GC_DDD_energy_files, 5, GC_X12_energy_files, 5, GC_DDD_bending_unified_10_mer_files, GC_X12_bending_unified_10_mer_files, True, 'GC_10_mer', 'sienna', 'darkorange', 'GC DDD', 'GC X12')
create_relevant_plots(GC_DDD_energy_files, 5, GC_X12_energy_files, 5, GC_DDD_bending_unified_13_mer_files, GC_X12_bending_unified_13_mer_files, True, 'GC_13_mer', 'sienna', 'darkorange', 'GC DDD', 'GC X12')
create_relevant_plots(GC_DDD_energy_files, 5, GC_X12_energy_files, 5, GC_DDD_bending_5_20_files, GC_X12_bending_5_20_files, False, 'GC_16_mer', 'sienna', 'darkorange', 'GC DDD', 'GC X12')

In [ ]:
#A function for extracting interaction energetic values of specific DNA groups from a data file
#Input: the path of the data file
#Output: a dataframe with the energetic values of the DNA groups (bases, sugars, phosphates), each contained within a specific column according to the correpsonding energy term (Coulomb, Van der Waals (LJ))
def Generate_Interaction_Energies_Groups_df(file_name):

    #Reading the data file as lists
    y = np.loadtxt(file_name, comments = ['@', '#'], unpack = True)
    lists = []
    for i in range(len(y)):
        lists.append(list(y[i]))

    #Inserting the lists to a dictionary, which would be converted to a dataframe with suitable column names, according to the energy term
    d = {}
    for i in range(len(lists)):
        d[i] = lists[i]
    df = pd.DataFrame(d)
    df.set_index(0, inplace=True)
    if len(df.columns) == 4:
        df.columns = ['Coul-SR', 'LJ-SR', 'Coul-14', 'LJ-14']
    elif len(df.columns) == 12:
        df.columns = ['PB Coul-SR', 'PB LJ-SR', 'PB Coul-14', 'PB LJ-14', 'PS Coul-SR', 'PS LJ-SR', 'PS Coul-14', 'PS LJ-14', 'PPh Coul-SR', 'PPh LJ-SR', 'PPh Coul-14', 'PPh LJ-14']
    
    #Returning the dataframe
    return df

#Reading the data files (of interaction energies of Hfq with the DNA groups) to a list of corresponding dataframes
Hfq_Proximal_B_DNA_CLOSE_files = ["/home_b/yoav/Hfq_Starting_Close_Protein_DNA_Contact/Hfq_Starting_Close_Protein_DNA_Contact_try_energy_groups_I.xvg",
                                  "/home_b/yoav/Hfq_Starting_Close_Protein_DNA_Contact/Hfq_Starting_Close_Protein_DNA_Contact_try_energy_groups_II.xvg",
                                  "/home_b/yoav/Hfq_Starting_Close_Protein_DNA_Contact/Hfq_Starting_Close_Protein_DNA_Contact_try_energy_groups_III.xvg"]
Hfq_Proximal_B_DNA_CLOSE_dfs = []
for file in Hfq_Proximal_B_DNA_CLOSE_files:
    Hfq_Proximal_B_DNA_CLOSE_dfs.append(Generate_Interaction_Energies_Groups_df(file))

#Concating all dataframes to a unified one
concat_Hfq_proximal_B_DNA_CLOSE_df = pd.DataFrame(columns = Hfq_Proximal_B_DNA_CLOSE_dfs[0].columns)
for proximal_B_DNA_df in Hfq_Proximal_B_DNA_CLOSE_dfs:
    concat_Hfq_proximal_B_DNA_CLOSE_df = pd.concat([concat_Hfq_proximal_B_DNA_CLOSE_df, proximal_B_DNA_df], axis = 0, ignore_index = True)

In [ ]:
#Transferring the unified dataframe to one containing all values in one column for the sake of illustration
concat_Hfq_proximal_B_DNA_CLOSE_df['PB_Sum-SR'] = concat_Hfq_proximal_B_DNA_CLOSE_df.copy()['PB Coul-SR'] + concat_Hfq_proximal_B_DNA_CLOSE_df.copy()['PB LJ-SR']
concat_Hfq_proximal_B_DNA_CLOSE_df['PS_Sum-SR'] = concat_Hfq_proximal_B_DNA_CLOSE_df.copy()['PS Coul-SR'] + concat_Hfq_proximal_B_DNA_CLOSE_df.copy()['PS LJ-SR']
concat_Hfq_proximal_B_DNA_CLOSE_df['PPh_Sum-SR'] = concat_Hfq_proximal_B_DNA_CLOSE_df.copy()['PPh Coul-SR'] + concat_Hfq_proximal_B_DNA_CLOSE_df.copy()['PPh LJ-SR']
relevant_cols = ['PB Coul-SR', 'PB LJ-SR', 'PB_Sum-SR', 'PS Coul-SR', 'PS LJ-SR', 'PS_Sum-SR', 'PPh Coul-SR', 'PPh LJ-SR', 'PPh_Sum-SR']
DNA_groups = ['Base'] * 3 + ['Sugar'] * 3 + ['Phosphate'] * 3
col_labels = ['Coulomb', 'LJ', 'Coulomb + LJ'] * 3
one_col_df = pd.DataFrame(columns = ['Energy', 'Col_Type', 'DNA_Group'])
for i_rc, rel_col in enumerate(relevant_cols):
    curr_rel_col_df = pd.DataFrame(columns = ['Energy', 'Col_Type', 'DNA_Group'])
    curr_rel_col_df['Energy'] = concat_Hfq_proximal_B_DNA_CLOSE_df.copy()[rel_col].values
    curr_rel_col_df['Col_Type'] = col_labels[i_rc]
    curr_rel_col_df['DNA_Group'] = DNA_groups[i_rc]
    one_col_df = pd.concat([one_col_df, curr_rel_col_df], axis = 0, ignore_index = True)

In [ ]:
#Summing up the Coulomb and LJ (VdW) terms
one_col_df_without_Summed = one_col_df.copy().loc[one_col_df.copy()['Col_Type'] != 'Coulomb + LJ']

In [ ]:
#Plotting a violin plots of the energies according to their term (Coulomb or LJ), coloring according to the DNA group (bases, sugars, phosphates) - panel S4A
plt.rcParams['pdf.fonttype'] = 42
sns.catplot(kind = 'violin', data = one_col_df_without_Summed, x = 'Col_Type', y = 'Energy', hue = 'DNA_Group')
plt.savefig('violinplots_B-DNA_groups_different_energy_terms_Without_Summed_Upload.pdf', format = 'pdf')

In [ ]:
#Reading energy data files of the interaction of Hfq's faces (distal, proximal, lateral/rim) with A-DNA (high salt), A-DNA (regular salt) and B-DNA
Hfq_A_Distal_High_Salt_Files = ["/home_b/yoav/Hfq_Distal_In_Front_Of_A_DNA_High_Salt/Hfq_Distal_A_DNA_GCX12_PR_High_Salt_energy_I.xvg", "/home_b/yoav/Hfq_Distal_In_Front_Of_A_DNA_High_Salt/Hfq_Distal_A_DNA_GCX12_PR_High_Salt_energy_II.xvg", "/home_b/yoav/Hfq_Distal_In_Front_Of_A_DNA_High_Salt/Hfq_Distal_A_DNA_GCX12_PR_High_Salt_energy_III.xvg"]
Hfq_A_Proximal_High_Salt_Files = ["/home_b/yoav/Hfq_A_DNA_GC_High_Salt/Hfq_A_DNA_GCX12_PR_High_Salt_energy_I.xvg", "/home_b/yoav/Hfq_A_DNA_GC_High_Salt/Hfq_A_DNA_GCX12_PR_High_Salt_energy_II.xvg", "/home_b/yoav/Hfq_A_DNA_GC_High_Salt/Hfq_A_DNA_GCX12_PR_High_Salt_energy_III.xvg"]
Hfq_A_Rim_High_Salt_Files = ["/home_b/yoav/Hfq_A_DNA_Rim_High_Salt/Hfq_A_DNA_GCX12_Rim_PR_High_Salt_energy_I.xvg", "/home_b/yoav/Hfq_A_DNA_Rim_High_Salt/Hfq_A_DNA_GCX12_Rim_PR_High_Salt_energy_II.xvg", "/home_b/yoav/Hfq_A_DNA_Rim_High_Salt/Hfq_A_DNA_GCX12_Rim_PR_High_Salt_energy_III.xvg"]
Hfq_A_Distal_Files = ["/home_b/yoav/Hfq_Distal_In_Front_Of_A_DNA/Hfq_Distal_A_DNA_GCX12_PR_energy_I.xvg", "/home_b/yoav/Hfq_Distal_In_Front_Of_A_DNA/Hfq_Distal_A_DNA_GCX12_PR_energy_II.xvg", "/home_b/yoav/Hfq_Distal_In_Front_Of_A_DNA/Hfq_Distal_A_DNA_GCX12_PR_energy_III.xvg"]
Hfq_A_Proximal_Files = ["/home_b/yoav/Hfq_A_DNA_GC/Hfq_A_DNA_GCX12.200ns.energy_I.xvg", "/home_b/yoav/Hfq_A_DNA_GC/Hfq_A_DNA_GCX12.200ns.energy_II.xvg", "/home_b/yoav/Hfq_A_DNA_GC/Hfq_A_DNA_GCX12.200ns.energy_III.xvg"]
Hfq_A_Rim_Files = ["/home_b/yoav/Hfq_A_DNA_Rim/Hfq_A_DNA_GCX12_Rim_energy_I.xvg", "/home_b/yoav/Hfq_A_DNA_Rim/Hfq_A_DNA_GCX12_Rim_energy_II.xvg", "/home_b/yoav/Hfq_A_DNA_Rim/Hfq_A_DNA_GCX12_Rim_energy_III.xvg"]
Hfq_B_Distal_Files = ["/home_b/yoav/Hfq_Distal_In_Front_Of_DNA_NMG/Hfq_DDD_RMG_24_Distal_PR_energy_I.xvg", "/home_b/yoav/Hfq_Distal_In_Front_Of_DNA_NMG/Hfq_DDD_RMG_24_Distal_PR_energy_II.xvg", "/home_b/yoav/Hfq_Distal_In_Front_Of_DNA_NMG/Hfq_DDD_RMG_24_Distal_PR_energy_III.xvg"]
Hfq_B_Proximal_Files = ["/home_b/yoav/Hfq_Starting_Close_Protein_DNA_Contact/Hfq_Starting_Close_Protein_DNA_Contact_try_PR_energy_I.xvg", "/home_b/yoav/Hfq_Starting_Close_Protein_DNA_Contact/Hfq_Starting_Close_Protein_DNA_Contact_try_PR_energy_II.xvg", "/home_b/yoav/Hfq_Starting_Close_Protein_DNA_Contact/Hfq_Starting_Close_Protein_DNA_Contact_try_PR_energy_II.xvg"]
Hfq_B_Rim_Files = ["/home_b/yoav/Hfq_DDD_NMG_24_Rim/Hfq_DDD_NMG_24_Rim_PR_energy_I.xvg", "/home_b/yoav/Hfq_DDD_NMG_24_Rim/Hfq_DDD_NMG_24_Rim_PR_energy_II.xvg", "/home_b/yoav/Hfq_DDD_NMG_24_Rim/Hfq_DDD_NMG_24_Rim_PR_energy_III.xvg"]

#For each set of energy files, converting them to corresponding dataframes
Hfq_A_Distal_High_Salt_Dfs = from_files_to_dfs(Hfq_A_Distal_High_Salt_Files)
Hfq_A_Proximal_High_Salt_Dfs = from_files_to_dfs(Hfq_A_Proximal_High_Salt_Files)
Hfq_A_Rim_High_Salt_Dfs = from_files_to_dfs(Hfq_A_Rim_High_Salt_Files)
Hfq_A_Distal_Dfs = from_files_to_dfs(Hfq_A_Distal_Files)
Hfq_A_Proximal_Dfs = from_files_to_dfs(Hfq_A_Proximal_Files)
Hfq_A_Rim_Dfs = from_files_to_dfs(Hfq_A_Rim_Files)
Hfq_B_Distal_Dfs = from_files_to_dfs(Hfq_B_Distal_Files)
Hfq_B_Proximal_Dfs = from_files_to_dfs(Hfq_B_Proximal_Files)
Hfq_B_Rim_Dfs = from_files_to_dfs(Hfq_B_Rim_Files)

In [ ]:
#A function for concating copies of energy dataframes to a unified one
#Input: list of dataframes
#Output: a unified/concat dataframe
def concat_copies(dfs):

    #Concating each dataframe to the unified one
    concat_df = pd.DataFrame(columns = dfs[0].columns)
    for df in dfs:
        concat_df = pd.concat([concat_df, df], axis = 0, ignore_index = True)
    
    #Returning the unified one
    return concat_df

#Concating the copies of each set of simulation (DNA type and Hfq face)
concat_A_HS_df_Distal = concat_copies(Hfq_A_Distal_High_Salt_Dfs); concat_A_HS_df_Proximal = concat_copies(Hfq_A_Proximal_High_Salt_Dfs); concat_A_HS_df_Rim = concat_copies(Hfq_A_Rim_High_Salt_Dfs)
concat_A_df_Distal = concat_copies(Hfq_A_Distal_Dfs); concat_A_df_Proximal = concat_copies(Hfq_A_Proximal_Dfs); concat_A_df_Rim = concat_copies(Hfq_A_Rim_Dfs)
concat_B_df_Distal = concat_copies(Hfq_B_Distal_Dfs); concat_B_df_Proximal = concat_copies(Hfq_B_Proximal_Dfs); concat_B_df_Rim = concat_copies(Hfq_B_Rim_Dfs)

#Concating all Hfq faces together according to the DNA type
concat_A_HS_dfs = [concat_A_HS_df_Distal, concat_A_HS_df_Proximal, concat_A_HS_df_Rim]
concat_A_dfs = [concat_A_df_Distal, concat_A_df_Proximal, concat_A_df_Rim]
concat_B_dfs = [concat_B_df_Distal, concat_B_df_Proximal, concat_B_df_Rim]

In [ ]:
#Concating all DNA types to a unified dataframe
all_dfs = [concat_A_HS_dfs, concat_A_dfs, concat_B_dfs]
faces = ['Distal', 'Proximal', 'Rim'] * 3
conditions = ['A_DNA_HS'] * 3 + ['A_DNA'] * 3 + ['B_DNA'] * 3
concat_all_df = pd.DataFrame(columns = list(all_dfs[0][0].columns) + ['Energy', 'Face', 'Condition'])
for i, curr_all_dfs in enumerate(all_dfs):
    for j, x in enumerate(curr_all_dfs):
        curr_x = x.copy()
        curr_x['Energy'] = curr_x.copy()['PD Coul-SR'] + curr_x.copy()['PD LJ-SR']
        curr_x['Face'] = faces[3 * i + j]
        curr_x['Condition'] = conditions[3 * i + j]
        concat_all_df = pd.concat([concat_all_df, curr_x], axis = 0, ignore_index = True)

In [ ]:
#Dissecting the unified dataframe according to DNA type
concat_all_df_A_HS = concat_all_df.copy().loc[concat_all_df.copy()['Condition'] == 'A_DNA_HS']
concat_all_df_A = concat_all_df.copy().loc[concat_all_df.copy()['Condition'] == 'A_DNA']
concat_all_df_B = concat_all_df.copy().loc[concat_all_df.copy()['Condition'] == 'B_DNA']

In [ ]:
#Dissecting the unified dataframe according to Hfq face
concat_all_df_Distal = concat_all_df.copy().loc[concat_all_df.copy()['Face'] == 'Distal']
concat_all_df_Proximal = concat_all_df.copy().loc[concat_all_df.copy()['Face'] == 'Proximal']
concat_all_df_Rim = concat_all_df.copy().loc[concat_all_df.copy()['Face'] == 'Rim']

In [ ]:
#A function for plotting 1D KDE plots comparing the interaction energies of Hfq either according to its face or to the DNA type
#Input: two energy dataframes to compare, the energies terms of both embodied by the dataframe columns, a string dictating whether the Hfq face or DNA type would be compared, the two faces/types to be compared, a title to the plots, x label, y label, plot limits
#Output: None
def plot_1D_KDE_plot(x_df, y_df, x_column, y_column, face_or_condition, x_face_or_condition, y_face_or_condition, title, x_label, y_label, lims):
    filt_x_df = x_df.copy().loc[x_df.copy()[face_or_condition] == x_face_or_condition].reset_index()
    filt_y_df = y_df.copy().loc[y_df.copy()[face_or_condition] == y_face_or_condition].reset_index()
    plt.rcParams['pdf.fonttype'] = 42
    fig, ax = plt.subplots(figsize = (8, 6))
    sns.kdeplot(ax = ax, x = filt_x_df[x_column], log_scale = False, cbar = True, fill = False, color = 'lightskyblue', alpha = 1, label = x_label, linewidth = 5)
    sns.kdeplot(ax = ax, x = filt_y_df[y_column], log_scale = False, cbar = True, fill = False, color = 'coral', alpha = 1, label = y_label, linewidth = 5)
    ax.set_xlabel('Interaction Energy [kJ/mole]', fontsize = 20)
    ax.set_ylabel('Density', fontsize = 20)
    ax.set_xlim(lims)
    ax.set_title(title)
    plt.legend()
    fig.savefig('KDE_Plot_1D_AA_' + x_label + '_' + x_column + '&' + y_label + '_' + y_column + 'Upload.pdf', format = 'pdf')

In [ ]:
#Comparing the interaction energies (Coulomb + LJ) of proximal and distal Hfq faces with A-DNA - panel 5A
plot_1D_KDE_plot(concat_all_df_Proximal, concat_all_df_Distal, 'Energy', 'Energy', 'Condition', 'A_DNA', 'A_DNA', 'A-DNA - Distal Hfq Vs. Proximal Hfq - All Energies', 'Hfq_Prox-A-DNA', 'Hfq_Dist-A-DNA', [-2000, 0])

In [ ]:
#Comparing the interaction energies (Coulomb) of proximal and distal Hfq faces with A-DNA - panel 5A
plot_1D_KDE_plot(concat_all_df_Proximal, concat_all_df_Distal, 'PD Coul-SR', 'PD Coul-SR', 'Condition', 'A_DNA', 'A_DNA', 'A-DNA - Distal Hfq Vs. Proximal Hfq - Coulomb', 'Hfq_Prox-A-DNA', 'Hfq_Dist-A-DNA', [-1600, 0])

In [ ]:
#Comparing the interaction energies (LJ) of proximal and distal Hfq faces with A-DNA - panel 5A
plot_1D_KDE_plot(concat_all_df_Proximal, concat_all_df_Distal, 'PD LJ-SR', 'PD LJ-SR', 'Condition', 'A_DNA', 'A_DNA', 'A-DNA - Distal Hfq Vs. Proximal Hfq - LJ', 'Hfq_Prox-A-DNA', 'Hfq_Dist-A-DNA',  [-1000, 0])

In [ ]:
#Comparing the interaction energies (Coulomb + LJ) of proximal and distal Hfq faces with B-DNA - panel 5B
plot_1D_KDE_plot(concat_all_df_Distal, concat_all_df_Proximal, 'Energy', 'Energy', 'Condition', 'B_DNA', 'B_DNA', 'B-DNA - Distal Hfq Vs. Proximal Hfq - All Energies', 'Hfq_Dist-B-DNA', 'Hfq_Prox-B-DNA', [-2000, 0])

In [ ]:
#Comparing the interaction energies (Coulomb) of proximal and distal Hfq faces with B-DNA - panel 5B
plot_1D_KDE_plot(concat_all_df_Distal, concat_all_df_Proximal, 'PD Coul-SR', 'PD Coul-SR', 'Condition', 'B_DNA', 'B_DNA', 'B-DNA - Distal Hfq Vs. Proximal Hfq - Coulomb', 'Hfq_Dist-B-DNA', 'Hfq_Prox-B-DNA', [-1600, 0])

In [ ]:
#Comparing the interaction energies (LJ) of proximal and distal Hfq faces with B-DNA - panel 5B
plot_1D_KDE_plot(concat_all_df_Distal, concat_all_df_Proximal, 'PD LJ-SR', 'PD LJ-SR', 'Condition', 'B_DNA', 'B_DNA', 'B-DNA - Distal Hfq Vs. Proximal Hfq - LJ', 'Hfq_Dist-B-DNA', 'Hfq_Prox-B-DNA',  [-1000, 0])

In [ ]:
#Plotting violin plots of interaction energies (Coulomb + LJ) of Hfq with the different DNA types, colored according to Hfq face - panel S4B
plt.rcParams['pdf.fonttype'] = 42
sns.violinplot(data = concat_all_df, x = 'Condition', y = 'Energy', hue = 'Face')
plt.savefig('Violin_Plot_Energy_DNA_Face_Upload.pdf', format = 'pdf')

In [ ]:
#Plotting violin plots of interaction energies (Coulomb + LJ) of Hfq with the different Hfq faces, colored according to DNA type - panel S4B
plt.rcParams['pdf.fonttype'] = 42
sns.violinplot(data = concat_all_df, x = 'Face', y = 'Energy', hue = 'Condition')
plt.savefig('Violin_Plot_Energy_Face_DNA_Upload.pdf', format = 'pdf')

In [ ]:
#Plotting violin plots of interaction energies (Coulomb) of Hfq with the different DNA types, colored according to Hfq's face - panel S4B
plt.rcParams['pdf.fonttype'] = 42
sns.violinplot(data = concat_all_df, x = 'Condition', y = 'PD Coul-SR', hue = 'Face')
plt.savefig('Violin_Plot_Coulomb_DNA_Face_Upload.pdf', format = 'pdf')

In [ ]:
#Plotting violin plots of interaction energies (Coulomb) of Hfq with the different Hfq faces, colored according to DNA type - panel S4B
plt.rcParams['pdf.fonttype'] = 42
sns.violinplot(data = concat_all_df, x = 'Face', y = 'PD Coul-SR', hue = 'Condition')
plt.savefig('Violin_Plot_Coulomb_Face_DNA_Upload.pdf', format = 'pdf')

In [ ]:
#Plotting violin plots of interaction energies (LJ) of Hfq with the different DNA types, colored according to Hfq's face - panel S4B
plt.rcParams['pdf.fonttype'] = 42
sns.violinplot(data = concat_all_df, x = 'Condition', y = 'PD LJ-SR', hue = 'Face')
plt.savefig('Violin_Plot_LJ_DNA_Face_Upload.pdf', format = 'pdf')

In [ ]:
#Plotting violin plots of interaction energies (LJ) of Hfq with the different Hfq faces, colored according to DNA type - panel S4B
plt.rcParams['pdf.fonttype'] = 42
sns.violinplot(data = concat_all_df, x = 'Face', y = 'PD LJ-SR', hue = 'Condition')
plt.savefig('Violin_Plot_LJ_Face_DNA_Upload.pdf', format = 'pdf')